In [11]:
import nibabel as nib
import pandas as pd
import numpy as np

def extract_values_from_coordinates(file_paths, coordinates, example_img_path):
    """
    Extract values from NIfTI files based on MNI coordinates and store them in a Pandas DataFrame.
    
    Parameters:
        file_paths (list): List of paths to the NIfTI files.
        coordinates (list): List of dictionaries containing MNI coordinates in the form {'x': x_val, 'y': y_val, 'z': z_val}.
        example_img_path (str): Path to an example NIfTI image to get the affine matrix from.
        
    Returns:
        pd.DataFrame: DataFrame with NIfTI file paths as rows and coordinates as columns.
    """
    from nimlab import datasets as nimds
    # Load the example NIfTI file to get the affine matrix
    if example_img_path is None:
        example_img = nimds.get_img("mni_icbm152")
    else:
        example_img = nib.load(example_img_path)
    affine = example_img.affine
    inv_affine = np.linalg.inv(affine)
    
    # Initialize an empty DataFrame
    df = pd.DataFrame(index=file_paths)
    
    for coord in coordinates:
        col_name = f"x{coord['x']}_y{coord['y']}_z{coord['z']}"
        values = []
        
        for file_path in file_paths:
            try:
                img = nib.load(file_path)
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                continue
            
            data = img.get_fdata()
            
            mni_coord = np.array([coord['x'], coord['y'], coord['z'], 1])
            voxel_coord = np.dot(inv_affine, mni_coord)
            i, j, k, _ = np.round(voxel_coord).astype(int)
            
            try:
                value = data[i, j, k]
            except IndexError:
                print(f"Invalid coordinate {coord} for file {file_path}")
                value = np.nan
            
            values.append(value)
        
        df[col_name] = values
    
    return df


file_paths = List of paths to the NIfTI files.

coordinates = List of dictionaries containing MNI coordinates in the form {'x': x_val, 'y': y_val, 'z': z_val}.

example_img_path = Path to an example NIfTI image to get the affine matrix from.

In [12]:
shared_root = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity'
file_pattern = '*.nii'
coordinates_list = [
    {'x': 0.9,'y': -70, 'z': -44},
    {'x': 17,'y': -99, 'z': 8.6},
    {'x': 25,'y': 7, 'z': -1},
    {'x': -5.7,'y': -2.7, 'z': -36},
    {'x': 23,'y': 63, 'z': 5.7},
    {'x': -51,'y': -19, 'z': 46}
]

In [13]:
from calvin_utils.file_utils.file_path_collector import glob_file_paths
from calvin_utils.file_utils.dataframe_util import
unsorted_df = extract_values_from_coordinates(file_paths=glob_file_paths(shared_root, file_pattern)['paths'].to_list(),
                                coordinates = coordinates_list,
                                example_img_path=None
                                )

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/*.nii


,x0.9_y-70_z-44,x17_y-99_z8.6,x25_y7_z-1,x-5.7_y-2.7_z-36,x23_y63_z5.7,x-51_y-19_z46
/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/116_vat_seed_compound_fMRI_efield_func_seed_T.nii,5.110469,-1.022281,-9.608702,-5.320343,4.541225,0.002873
/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/1203_vat_seed_compound_fMRI_efield_func_seed_T.nii,2.839277,-4.369250,5.530316,-9.048473,6.804471,-2.252330
/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/139_vat_seed_compound_fMRI_efield_func_seed_T.nii,3.210808,-2.762391,-6.420975,-7.199178,4.717632,0.378458
/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/147_vat_seed_compound_fMRI_efield_func_seed_T.nii,1.555684,-4.117804,0.669473,-9.443657,5.444838,-0.530062
/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/135_vat_seed_compound_fMRI_efield_func_seed_T.nii,5.415641,-0.649956,-7.719434,-6.368051,4.324275,-0.550868
/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/101_vat_seed_compound_fMRI_efield_func_seed_T.nii,4.673572,0.028204,-13.967035,-2.649197,3.013867,2.289633
/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/150_vat_seed_compound_fMRI_efield_func_seed_T.nii,3.389582,-6.091383,14.014598,-11.378032,7.917304,-6.574064
/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/122_vat_seed_compound_fMRI_efield_func_seed_T.nii,2.591133,-4.686292,1.765084,-8.717899,7.063189,-1.744410
/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/1202_vat_seed_compound_fMRI_efield_func_seed_T.nii,4.837078,1.108034,-17.811454,0.669343,2.294574,2.600107
/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/146_vat_seed_compound_fMRI_efield_func_seed_T.nii,4.617518,-1.357736,-8.396539,-5.676133,2.261734,-1.815690
